In [48]:
import csv
import itertools
import pandas as pd
import numpy as np

In [7]:
# Connection table between nodes
connections = pd.read_csv("Brain1Connections.csv", header=None)

In [8]:
# Positions of nodes (X,Y,Z)
positions = pd.read_csv("Brain1Positions.csv", header=None, sep=";")
positions[0][0] = positions[0][0][:-1]

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Convert data to numerical value
positions[0] = positions[0].astype(float)

In [10]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [11]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [12]:
# Calculate distance between 2 points in Euclidean space

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [13]:
Euclidean_dist(0,1)

19.49822697213286

In [14]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p):
    neighbours = []
    
    for i in range(83):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [15]:
n = get_neighbours(1)
n

[0, 4, 5, 6, 7, 17, 18, 22, 23, 24, 33, 34, 35, 36, 37, 47, 48, 82]

In [16]:
# Add a key-value pair to the existing dictionary
def add_element(dict, key, value):
    if key not in dict:
        dict[key] = []
    dict[key].append(value)

# Returns with a dictionary. Key is the number of the node and 
# the value is the distant between the current node(act) and its neighbour.
# The list is ascendant by the distance.
def get_dist(act, neighbours):
    dist = {}
    
    for i in neighbours:
        add_element(dist, i, Euclidean_dist(act, i))
    
    dist = sorted(dist.items(), key=lambda x: x[1])
    
    return dist

In [17]:
dict = get_dist(1, n)
print(dict)
dict[5]

[(4, [15.344590572116752]), (0, [19.49822697213286]), (6, [25.798750941939833]), (5, [36.720471587727296]), (36, [41.395190709541474]), (33, [42.197853587912384]), (35, [45.286121681769224]), (37, [48.96544753911202]), (7, [52.988761474508685]), (48, [66.83591484552962]), (34, [67.55024761276022]), (47, [70.24085215312306]), (82, [79.30335523867505]), (24, [90.88549439891962]), (18, [107.84129652866407]), (23, [112.0130201522961]), (17, [114.70425154607828]), (22, [125.6035193958912])]


(33, [42.197853587912384])

In [18]:
# Calculates the full lenght of the path through the nodes.
def path_dist(g):
    dist = 0
    
    for i in range(len(g)-1):
        dist = dist + Euclidean_dist(g[i], g[i+1])
    
    return dist

In [19]:
path_dist([0, 3, 44, 52, 11, 12, 53, 76, 9])

170.86388025914366

# Algorithm 1.

In [20]:
# The algorithm chooses the node which one is the closest to the current node and they are neighbours.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy1(p, z):
    ready = False
    routing = []
    routing.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing.append(z)
            return routing
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            act = neighbours[0]
            routing.append(act)
            
        elif len(neighbours) > 1:
            dist = get_dist(act, neighbours)
            
            # print("Neighbours: ", neighbours)
            # print("DIST: ", dist)
            # print()
            
            not_found = False
            for i in dist:
                if i[0] not in routing:
                    act = i[0]
                    routing.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                act = dist[0][0]
                routing.append(act)

In [21]:
routing_map_1 = []

# Add the path of p->z to the routing table 
def greedy1_algorithm(p,z):
    path = greedy1(p,z)
    if path not in routing_map_1:
        routing_map_1.append(path)

In [22]:
# Iterate throught all of the nodes and run the greedy1 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy1_algorithm(i,j)

In [23]:
# Calculate the average of the nodes of the paths
sum_nodes1 = 0
for i in range(6806):
    sum_nodes1 = sum_nodes1 + len(routing_map_1[i])
    
avr_nodes1 = sum_nodes1 / 6806
avr_nodes1

8.070232148104614

In [24]:
# Calculate the average distance of the paths
sum_dist1 = 0
for i in range(6806):
    sum_dist1 = sum_dist1 + path_dist(routing_map_1[i])
    
avr_dist1 = sum_dist1 / 6806
avr_dist1

162.67517508376523

# Algorithm 2.

In [25]:
# The algorithm chooses the neighbour node which one is the closest to the end node.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy2(p, z):
    ready = False
    routing = []
    routing.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing.append(z)
            return routing
        
        # if 'act' has no neighbours 
        elif len(neighbours) == 0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            act = neighbours[0]
            routing.append(act)
            
        elif len(neighbours) > 1:
            # Calculates the distances of the neighbour nodes from the end node
            dist = get_dist(z, neighbours)
            
            # print("Neighbours: ", neighbours)
            # print("DIST: ", dist)
            # print()
            
            not_found = False
            for i in dist:
                if i[0] not in routing:
                    act = i[0]
                    routing.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                act = dist[0][0]
                routing.append(act)

In [26]:
routing_map_2 = []

# Add the path of p->z to the routing table 
def greedy2_algorithm(p,z):
    path = greedy2(p,z)
    if path not in routing_map_2:
        routing_map_2.append(path)

In [27]:
# Iterate throught all of the nodes and run the greedy2 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy2_algorithm(i,j)

In [28]:
# Calculate the average of the nodes of the paths
sum_nodes2 = 0
for i in range(6806):
    sum_nodes2 = sum_nodes2 + len(routing_map_2[i])
    
avr_nodes2 = sum_nodes2 / 6806
avr_nodes2

3.0113135468704084

In [29]:
# Calculate the average distance of the paths
sum_dist2 = 0
for i in range(6806):
    sum_dist2 = sum_dist2 + path_dist(routing_map_2[i])
    
avr_dist2 = sum_dist2 / 6806
avr_dist2

84.29096240285445

# Algorithm 3.

In [30]:
# The algorithm chooses the neighbour node, through the distance to the end node from 'p' is the shortest.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy3(p, z):
    ready = False
    routing = []
    routing.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing.append(z)
            return routing
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            act = neighbours[0]
            routing.append(act)
            
        elif len(neighbours) > 1:
            # distance from 'p' to the neighbours
            dist1 = get_dist(act, neighbours)
            # distance from the neighbours to the end node
            dist2 = get_dist(z, neighbours)
            dist = {}
            
            # Calculate the distance from 'p' through the neighbours to the end node
            for i in range(len(neighbours)):
                for j in range(len(neighbours)):
                    if dist1[i][0]==dist2[j][0]:
                        val1 = dist1[i][1]
                        val2 = dist2[j][1]
                        add_element(dist, dist1[i][0], val1[0]+val2[0])
            dist = sorted(dist.items(), key=lambda x: x[1])
            
            # print("Neighbours: ", neighbours)
            # print("DIST: ", dist)
            # print()
            
            not_found = False
            for i in dist:
                if i[0] not in routing:
                    act = i[0]
                    routing.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                act = dist[0][0]
                routing.append(act)

In [31]:
routing_map_3 = []

# Add the path of p->z to the routing table 
def greedy3_algorithm(p,z):
    path = greedy3(p,z)
    if path not in routing_map_3:
        routing_map_3.append(path)

In [32]:
# Iterate throught all of the nodes and run the greedy3 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy3_algorithm(i,j)

In [33]:
# Calculate the average of the nodes of the paths
sum_nodes3 = 0
for i in range(6806):
    sum_nodes3 = sum_nodes3 + len(routing_map_3[i])
    
avr_nodes3 = sum_nodes3 / 6806
avr_nodes3

3.564942697619747

In [34]:
# Calculate the average distance of the paths
sum_dist3 = 0
for i in range(6806):
    sum_dist3 = sum_dist3 + path_dist(routing_map_3[i])
    
avr_dist3 = sum_dist3 / 6806
avr_dist3

78.99626993739102

## Algorithm 1:
    - Average node number of paths:
        8.070232148104614
    - Average path lenght:
        162.67517508376523
    
## Algorithm 2:
    - Average node number of paths:
        3.0113135468704084
    - Average path lenght: 
        84.29096240285445
    
## Algorithm 3:
    - Average node number of paths: 
        3.564942697619747 
    - Average path lenght: 
        78.99626993739102

In [44]:
routing_map_2

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 8, 9],
 [0, 17, 10],
 [0, 3, 11],
 [0, 7, 12],
 [0, 34, 10, 13],
 [0, 34, 19, 14],
 [0, 8, 15],
 [0, 30, 16],
 [0, 17],
 [0, 18],
 [0, 17, 19],
 [0, 23, 20],
 [0, 23, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 24, 25],
 [0, 40, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 31, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 40, 39],
 [0, 40],
 [0, 3, 44, 41],
 [0, 3, 47, 42],
 [0, 2, 3, 43],
 [0, 3, 44],
 [0, 3, 47, 45],
 [0, 35, 46],
 [0, 3, 47],
 [0, 7, 48],
 [0, 7, 49],
 [0, 35, 50],
 [0, 17, 51],
 [0, 3, 52],
 [0, 35, 76, 53],
 [0, 34, 10, 54],
 [0, 34, 60, 55],
 [0, 34, 50, 56],
 [0, 34, 50, 57],
 [0, 17, 58],
 [0, 17, 58, 59],
 [0, 17, 60],
 [0, 23, 61],
 [0, 23, 62],
 [0, 23, 63],
 [0, 23, 64],
 [0, 82, 64, 65],
 [0, 82, 80, 66],
 [0, 82, 81, 67],
 [0, 38, 81, 68],
 [0, 82, 70, 69],
 [0, 82, 70],
 [0, 82, 70, 71],
 [0, 82, 72],
 [0, 82, 72, 73],
 [0, 38, 77, 74],
 [0, 34

In [45]:
routing_map_3

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 5, 9],
 [0, 35, 10],
 [0, 3, 11],
 [0, 35, 38, 12],
 [0, 35, 37, 34, 75, 55, 13],
 [0, 34, 75, 55, 14],
 [0, 5, 15],
 [0, 33, 16],
 [0, 17],
 [0, 18],
 [0, 36, 19],
 [0, 37, 20],
 [0, 37, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 40, 25],
 [0, 27, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 33, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 40, 39],
 [0, 40],
 [0, 3, 44, 41],
 [0, 3, 44, 41, 42],
 [0, 3, 43],
 [0, 3, 44],
 [0, 3, 44, 41, 45],
 [0, 3, 52, 44, 41, 46],
 [0, 3, 47],
 [0, 3, 48],
 [0, 3, 52, 48, 49],
 [0, 38, 50],
 [0, 35, 10, 51],
 [0, 3, 52],
 [0, 35, 76, 53],
 [0, 35, 76, 54],
 [0, 38, 34, 75, 55],
 [0, 38, 76, 56],
 [0, 38, 78, 57],
 [0, 35, 34, 58],
 [0, 38, 78, 59],
 [0, 35, 34, 60],
 [0, 34, 20, 61],
 [0, 38, 34, 20, 62],
 [0, 38, 82, 63],
 [0, 38, 82, 64],
 [0, 38, 78, 80, 65],
 [0, 38, 79, 80, 66],
 [0, 38, 81, 67],
 [0, 3, 44, 41, 42, 45, 77, 81, 68],

In [51]:
with open('output.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_2[index]] for index in range(0, len(routing_map_3)))